In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/multi_turn_rewrite/chinese/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time
import numpy as np

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def align_pad(li):
  max_len = max([len(sent) for sent in li])
  for sent in li:
    if len(sent) < max_len:
      sent += [0] * (max_len - len(sent))

In [0]:
def parse_fn(text):
  h1, h2, q = text.rstrip().split('|')
  char2idx_fn = lambda x: [params['char2idx'].get(c, len(params['char2idx'])) for c in list(x)]
  h1, h2, q = char2idx_fn(h1), char2idx_fn(h2), char2idx_fn(q)
  align_pad([h1, h2])
  return {'query': np.reshape(q, (1, len(q))),
          'history':np.reshape([h1, h2], (1, 2, len(h1)))
  }

In [6]:
params = {'export_dir': '../model/baseline_lstm_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])
print(latest)

predict_fn = tf.contrib.predictor.from_saved_model(latest)

../model/baseline_lstm_export/1578359027
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/baseline_lstm_export/1578359027/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  print('A:', ' '.join([params['idx2char'].get(idx, '<unk>') for idx in predictions['output'][0] if (idx != 0 and idx != 2)]))
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:c语言会吗|一个集成计算机语言编写环境|教教我
A: 教 教 我 c 语 言
Cost 0.37 sec

Input:福州明天的天气是多云25度到35度微风小于3级|热啊|今天最低温度多少
A: 福 州 今 天 最 低 温 度 多 少
Cost 0.08 sec

Input:你知道jamc吗|哈哈哈知道啦|是什么
A: j a m c 是 什 么
Cost 0.07 sec

Input:人间失格吧|第一个故事特别棒 但整体书籍太深奥 很难看下去|挺好看的
A: 人 间 失 格 挺 好 看 的
Cost 0.08 sec

Input:有PS的下载链接吗|优酷上有啊|我要百度云的
A: 我 要 百 度 云 的
Cost 0.08 sec

Input:听说过马兜铃酸吗|不知道|这有毒不能吃
A: 马 兜 铃 酸 有 毒 不 能 吃
Cost 0.08 sec

Input:怎么长到一米八|身高是硬伤啊|我不低
A: 我 米 八 不 低
Cost 0.07 sec

Input:不想吃火锅|吃吧带你去成都|没钱去
A: 没 钱 去 成 都
Cost 0.05 sec

Input:你知道初音未来吗|我认得她的声音唱的歌挺有风格哒|她有哪些歌
A: 初 音 未 来 有 哪 些 歌
Cost 0.08 sec

Input:明天呢|嗯 明天|下雨
A: 明 天 下 雨
Cost 0.05 sec

Input:我是梅州的|梅州在哪里|应该是广东省东北部吧
A: 梅 州 应 该 是 广 东 省 东 北 部 吧
Cost 0.10 sec

Input:请问现在微软的总裁是谁|萨蒂亚纳德拉啊大家都知道的吧|我怎么不知道
A: 我 怎 么 不 知 道 萨 蒂 亚 纳 德 拉
Cost 0.10 sec

Input:什么内容|厨房收纳攻略感觉超实用的|给我看看
A: 给 我 看 看 厨 房 收 纳 攻 略
Cost 0.09 sec

Input:莫愁喜欢谁啊|哪个版本的神雕啊|刘亦菲啊
A: 莫 愁 喜 欢 刘 亦 菲 啊
Cost 0.09 sec

